In [ ]:
from pycorenlp import StanfordCoreNLP 
import TreeKernel as tk
import TreeBuild as tb
import numpy as np
import pandas as pd
import pickle

nlp = StanfordCoreNLP('http://localhost:9000')

In [ ]:
def _getNLPToks_(rawSentence):
    try:
        output = nlp.annotate(rawSentence, properties={
            'annotators': 'tokenize,ssplit,pos,parse',
            'outputFormat': 'json'
        })
    except UnicodeDecodeError:
        sentence = unidecode(rawSentence)
        output = nlp.annotate(sentence, properties={
            'annotators': 'tokenize,ssplit,pos,parse',
            'outputFormat': 'json'
        })
    tokens = output['sentences'][0]['tokens']
    parse = output['sentences'][0]['parse'].split("\n")
    return {
        'toks':tokens, 'parse':parse
    }


In [ ]:
feature_vect = {}

with open('input/stanfordData_train1.nlp', 'rb') as handle:
    count = 0
    while True:
        if (count % 10000 == 0):
            print(count)
        try:
            unit = pickle.load(handle)
            # ST Syntax score
            (rscore_st, nscore_st) = tk._CollinsDuffy_(unit['q1']['parse'], unit['q2']['parse'], 0.8, 1, 0)

            feature_vect[unit['id']] = {
                'id':unit['id'], 
                'cdNorm_st':nscore_st
            }
            
        except EOFError:
            break
        except:
            print("Quote error on: %d", unit['id'])
            
            q1_stanford = _getNLPToks_(unit['q1']['raw'].replace('"','').replace("'",''))
            q2_stanford = _getNLPToks_(unit['q2']['raw'].replace('"','').replace("'",''))

            tree_1 = tb.tree()
            tree_2 = tb.tree()

            # Generate a tree structure
            tb._generateTree_(q1_stanford['parse'], tree_1)
            tb._generateTree_(q2_stanford['parse'], tree_2)

            # Flip the trees
            tb._flipTree_(tree_1)
            tb._flipTree_(tree_2)

            
            (rscore_st, nscore_st) = tk._CollinsDuffy_(tree_1, tree_2, 0.8, 1, 0)

            feature_vect[unit['id']] = {
                'id':unit['id'], 
                'cdNorm_st':nscore_st
            }
            
            print("Quote error resolved")
            pass 

        count += 1

In [ ]:
df_feature = pd.DataFrame.from_dict(feature_vect)
df_feature = df_feature.transpose()

In [ ]:
df_feature[['id']] = df_feature[['id']].astype(int)

In [ ]:
df_feature.to_csv('input/collins_duffy_8e-1.csv', index=False)

In [ ]:
df_train = pd.read_csv('input/train.csv')

In [ ]:
df_train.ix[14343]


In [ ]:
df_feature.ix[14343]